# 1. Import

In [6]:
import MetaTrader5 as mt5
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime as dt


from config.config import MT5_LOGIN, MT5_PASSWORD, MT5_SERVER, PAIR, RISK_FREE_RATE, TIMEFRAME, START_DATE, END_DATE, INITIAL_CAPITAL, RISK_MODE, RISK_PER_TRADE, COMMISSION_PER_LOT, DONCHIAN_LOOKBACK
from backtest.runner_v1 import run_backtest_for_symbol as run_backtest_for_symbol_v1
from backtest.runner_v2 import run_backtest_for_symbol as run_backtest_for_symbol_v2
from exporation.plotting_utils import plot_equity_and_dd, plot_trade_distribution_and_side_pnl
from optimization.grind_search import grind_search_parameters

from exporation.plotting import CandlePlot 

# 2. Backtest the strategy

## 2.1 Start MT5 to get data

In [2]:
mt5.initialize()
ok = mt5.login(MT5_LOGIN, MT5_PASSWORD, MT5_SERVER)
print("MT5 initialization:", ok)

MT5 initialization: True


## 2.2 Run backtest for version 1 of the strategy

**Backtest setup:**
- **Pair:** BTCUSD  
- **Periods:**  
  - In-sample: 2018-03-01 → 2023-12-31  
  - Out-of-sample: 2024-01-01 → 2025-11-10 (to evaluate the performance of optimized parameters)

**Strategy concept:**  
The Donchian Breakout strategy identifies breakouts from a defined price range.

- **Buy signal:**  
  Triggered when the closing price breaks **above** the Donchian high.  
  Stop loss is placed at the Donchian low.

- **Sell signal:**  
  Triggered when the closing price breaks **below** the Donchian low.  
  Stop loss is placed at the Donchian high.

Positions are maintained until an **opposite signal** occurs, at which point all the current trades are closed and a new one is opened in the reverse direction.

**Backtest result:**
- The strategy is profitable, but its return distribution is **right-skewed** - most trades result in **small losses (around –1%)**, while a few large winning trades drive overall profitability by capturing strong upward or downward trends.  
- Consequently, during sideways markets, the strategy tends to generate frequent false signals, which can lead to **significant drawdowns over a short period** if too many trades are triggered.

***Therefore, to reduce the drawdown, I modify the strategy in Version 2 to allow only one open position at a time.***


In [24]:
res_v1 = run_backtest_for_symbol_v1(
    pair=PAIR,
    timeframe=TIMEFRAME,
    start_date=START_DATE,
    end_date=END_DATE,
    initial_capital=INITIAL_CAPITAL,
    risk_per_trade=RISK_PER_TRADE,
    risk_mode=RISK_MODE,
    commission_per_lot=COMMISSION_PER_LOT,
    lookback=DONCHIAN_LOOKBACK,
)

print(f"\n=== {PAIR} ===")
display(res_v1["report_df"].T)

fig_equity = plot_equity_and_dd(res_v1["balance_daily"], res_v1["dd_pct"])
fig_equity.update_layout(title_text=f"Equity & DD — {PAIR}")
fig_equity.show()

fig_trade = plot_trade_distribution_and_side_pnl(res_v1["trades"])
fig_trade.update_layout(title_text=f"Trade Distribution & Side PnL — {PAIR}")
fig_trade.show()

Getting data of BTCUSD_16385 successfully 

=== BTCUSD ===


,0
Start date,2018-03-01 00:00:00
End date,2023-12-01 00:00:00
Duration (days),2101
Trades,2155
Equity Final ($),100895.74
Equity Peak ($),102240.96
Net Profit ($),95895.74
Return (%),1917.91
Return (annual - %),68.6
Return (monthly - %),4.45


## 2.3 Run backtest for version 2 of the strategy

In Version 2 of the strategy, several improvements in performance can be observed:

- The overall return decreased significantly compared to Version 1 (from +1,917% to +258%); however, the annualized return remains around 25%, indicating a balanced trade-off between profitability and stability.  
- The **drawdown has been effectively controlled**, staying around **–13%**, much lower than the –75% observed in Version 1. The **equity curve is smoother**, with no sharp or prolonged declines, showing improved consistency.  
- However, the **trading frequency dropped significantly**, with only **313 trades over six years** (~less than one trade per week). While this helps reduce risk and overtrading, it may also cause the strategy to miss certain short-term opportunities.


In [23]:
res_v2 = run_backtest_for_symbol_v2(
    pair=PAIR,
    timeframe=TIMEFRAME,
    start_date=START_DATE,
    end_date=END_DATE,
    initial_capital=INITIAL_CAPITAL,
    risk_per_trade=RISK_PER_TRADE,
    risk_mode=RISK_MODE,
    commission_per_lot=COMMISSION_PER_LOT,
    lookback=DONCHIAN_LOOKBACK,
)

print(f"\n=== {PAIR} ===")
display(res_v2["report_df"].T)

fig_equity = plot_equity_and_dd(res_v2["balance_daily"], res_v2["dd_pct"])
fig_equity.update_layout(title_text=f"Equity & DD — {PAIR}")
fig_equity.show()

fig_trade = plot_trade_distribution_and_side_pnl(res_v2["trades"])
fig_trade.update_layout(title_text=f"Trade Distribution & Side PnL — {PAIR}")
fig_trade.show()

Getting data of BTCUSD_16385 successfully 

=== BTCUSD ===


,0
Start date,2018-03-01 00:00:00
End date,2023-12-01 00:00:00
Duration (days),2101
Trades,313
Equity Final ($),17919.58
Equity Peak ($),18557.2
Net Profit ($),12919.58
Return (%),258.39
Return (annual - %),24.85
Return (monthly - %),1.87


# 3. Grind search to choose best parameter

I define the selection criterion for the optimized parameter as follows:  
A parameter value is considered optimal when small variations around it do not cause significant changes in key performance metrics.
Based on this criterion, there are two stable regions that satisfy these conditions:  
- **Lookback 60–75**  
- **Lookback 160–200**  

**Therefore, I select 72** as the optimized parameter, as it lies within a stable region that balances responsiveness and robustness.

In [7]:
lookbacks_list = list(range(10, 201, 5))

grind_df, figs = grind_search_parameters(
    pairs=PAIR,
    timeframe=TIMEFRAME,
    start_date=START_DATE,
    end_date=END_DATE,
    lookbacks=lookbacks_list,
    initial_capital=INITIAL_CAPITAL,
    risk_per_trade=RISK_PER_TRADE,
    risk_mode=RISK_MODE,
    commission_per_lot=COMMISSION_PER_LOT,
    backtest_fn=run_backtest_for_symbol_v2,
    plot_charts=True,
)

print(grind_df.head())
figs["BTCUSD"].show()

Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting data of BTCUSD_16385 successfully 
Getting dat

# 4. Check the best parameter performance

In [31]:
OPTIMIZE_LOOKBACK = 72
TEST_START = dt(2024, 1, 1)
TEST_END = dt(2025, 11, 10)

## 4.1 Train data  

In [32]:
res_opt_train = run_backtest_for_symbol_v2(
    pair=PAIR,
    timeframe=TIMEFRAME,
    start_date=START_DATE,
    end_date=END_DATE,
    initial_capital=INITIAL_CAPITAL,
    risk_per_trade=RISK_PER_TRADE,
    risk_mode=RISK_MODE,
    commission_per_lot=COMMISSION_PER_LOT,
    lookback=OPTIMIZE_LOOKBACK,
)

print(f"\n=== {PAIR} ===")
display(res_opt_train["report_df"].T)

fig_equity = plot_equity_and_dd(res_opt_train["balance_daily"], res_opt_train["dd_pct"])
fig_equity.update_layout(title_text=f"Equity & DD — {PAIR}")
fig_equity.show()

fig_trade = plot_trade_distribution_and_side_pnl(res_opt_train["trades"])
fig_trade.update_layout(title_text=f"Trade Distribution & Side PnL — {PAIR}")
fig_trade.show()

Getting data of BTCUSD_16385 successfully 

=== BTCUSD ===


,0
Start date,2018-03-01 00:00:00
End date,2023-12-01 00:00:00
Duration (days),2101
Trades,449
Equity Final ($),22446.83
Equity Peak ($),23754.9
Net Profit ($),17446.83
Return (%),348.94
Return (annual - %),29.83
Return (monthly - %),2.2


## 4.2 Test data

In [33]:
res_opt_test = run_backtest_for_symbol_v2(
    pair=PAIR,
    timeframe=TIMEFRAME,
    start_date=TEST_START,
    end_date=TEST_END,
    initial_capital=INITIAL_CAPITAL,
    risk_per_trade=RISK_PER_TRADE,
    risk_mode=RISK_MODE,
    commission_per_lot=COMMISSION_PER_LOT,
    lookback=OPTIMIZE_LOOKBACK,
)

print(f"\n=== {PAIR} ===")
display(res_opt_test["report_df"].T)

fig_equity = plot_equity_and_dd(res_opt_test["balance_daily"], res_opt_test["dd_pct"])
fig_equity.update_layout(title_text=f"Equity & DD — {PAIR}")
fig_equity.show()

fig_trade = plot_trade_distribution_and_side_pnl(res_opt_test["trades"])
fig_trade.update_layout(title_text=f"Trade Distribution & Side PnL — {PAIR}")
fig_trade.show()

Getting data of BTCUSD_16385 successfully 

=== BTCUSD ===


,0
Start date,2024-01-01 00:00:00
End date,2025-11-10 00:00:00
Duration (days),679
Trades,150
Equity Final ($),6951.55
Equity Peak ($),7693.39
Net Profit ($),1951.55
Return (%),39.03
Return (annual - %),19.39
Return (monthly - %),1.49


In [34]:
report_train = res_opt_train["report_df"].T.copy()
report_test = res_opt_test["report_df"].T.copy()
report_train.columns = ["Train"]
report_test.columns = ["Test"]
report_compare = pd.concat([report_train, report_test], axis=1)
display(report_compare)

,Train,Test
Start date,2018-03-01 00:00:00,2024-01-01 00:00:00
End date,2023-12-01 00:00:00,2025-11-10 00:00:00
Duration (days),2101,679
Trades,449,150
Equity Final ($),22446.83,6951.55
Equity Peak ($),23754.9,7693.39
Net Profit ($),17446.83,1951.55
Return (%),348.94,39.03
Return (annual - %),29.83,19.39
Return (monthly - %),2.2,1.49
